In [2]:
import pandas as pd
import numpy as np
import datetime as dt

# Prepare the dataset

To prepare for the buyer features, we have to use the log dataset, which contains the behaviours of around 5000 frequent usersover 6 months.

In [37]:
df = pd.read_csv('product_log.csv')

# train test split

In [70]:
# ignore 2013-09, which serves as outcomes in test dataset
df['vmonth'] = df.vtime.str[: 7]
df_train = df[df.vmonth != '2013-09']

In [120]:
df_train.loc[:, 'vtime'] = df_train.vtime.apply(pd.to_datetime)

/var/folders/qq/cy_ydgl96ln1wcgv8d9hyw5w0000gn/T/ipykernel_1517/3199185038.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.loc[:, 'vtime'] = df_train.vtime.apply(pd.to_datetime)


In [71]:
# keep 2013-08 as outcome, and use data before that to create features
train = df_train[df_train.vmonth != '2013-08'].reset_index(drop=True)
out = df_train[df_train.vmonth == '2013-08'].reset_index(drop=True)

# seller to user data

In [38]:
%store -r user_id
user_id = list(user_id)
seller_id = list(set(df.seller_id))

In [123]:
# filter training data to consider only the buyer who will purchase any item next month
train = train[train.user_id.isin(user_id)]
train = train.reset_index(drop=True) 

In [39]:
index = pd.MultiIndex.from_product([user_id, seller_id], names=['user_id', 'seller_id'])

In [124]:
# create index based on the first state prediction
index = pd.MultiIndex.from_product([user_id, seller_id], names=['user_id', 'seller_id'])
b_s = pd.DataFrame(index=index)

## historical

In [125]:
# group training dataset by user_id, seller_id, action, for historical features
group = train.groupby(['user_id', 'seller_id', 'action']).count()
group = group.reset_index()

In [126]:
# purchase
to_merge = group[group.action == 'alipay'][['user_id', 'seller_id', 'item_id']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'purchase'})

In [127]:
# click
to_merge = group[group.action == 'click'][['user_id', 'seller_id', 'item_id']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'click'})

In [128]:
# cart
to_merge = group[group.action == 'cart'][['user_id', 'seller_id', 'item_id']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'cart'})

In [129]:
b_s['click_buy_ratio'] = b_s.purchase / b_s.click
b_s['cart_buy_ratio'] = b_s.purchase / b_s.cart

### last month

In [130]:
# group training dataset by user_id, seller_id, action, but only consider July
last_month = train[train.vmonth == '2013-07'].groupby(['user_id', 'seller_id', 'action']).count()[['item_id']]
last_month = last_month.reset_index()

In [131]:
to_merge = last_month[last_month.action == 'alipay'][['user_id', 'seller_id', 'item_id']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'purchase_last_m'})

In [132]:
to_merge = last_month[last_month.action == 'click'][['user_id', 'seller_id', 'item_id']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'click_last_m'})

In [133]:
to_merge = last_month[last_month.action == 'cart'][['user_id', 'seller_id', 'item_id']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'cart_last_m'})

In [134]:
b_s['click_buy_ratio_last_m'] = b_s.purchase_last_m / b_s.click_last_m
b_s['cart_buy_ratio_last_m'] = b_s.purchase_last_m / b_s.cart_last_m

### recent

In [135]:
recent = train[train.vtime > dt.datetime(2013, 7, 20)].groupby(['user_id', 'seller_id', 'action']).count()[['item_id']].reset_index()

In [136]:
to_merge = recent[recent.action == 'alipay'][['user_id', 'seller_id', 'item_id']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'purchase_recent'})

In [137]:
to_merge = recent[recent.action == 'click'][['user_id', 'seller_id', 'item_id']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'click_recent'})

In [138]:
to_merge = recent[recent.action == 'cart'][['user_id', 'seller_id', 'item_id']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'cart_recent'})

In [139]:
b_s['click_buy_ratio_recent'] = b_s.purchase_recent / b_s.click_recent
b_s['cart_buy_ratio_recent'] = b_s.purchase_recent / b_s.cart_recent

In [140]:
b_s= b_s.fillna(0)

### last

In [141]:
to_merge = train[train.action == 'alipay'].groupby(['user_id', 'seller_id']).max()[['vtime']]
to_merge['last_purchase'] = dt.datetime(2013, 8, 1) - to_merge.vtime
to_merge['last_purchase'] = to_merge.last_purchase.apply(lambda x: x.days)
to_merge = to_merge.reset_index()
b_s = b_s.merge(to_merge[['user_id', 'seller_id', 'last_purchase']], how='left', on=['user_id', 'seller_id'])

In [142]:
to_merge = train[train.action == 'click'].groupby(['user_id', 'seller_id']).max()[['vtime']]
to_merge['last_click'] = dt.datetime(2013, 8, 1) - to_merge.vtime
to_merge['last_click'] = to_merge.last_click.apply(lambda x: x.days)
to_merge = to_merge.reset_index()
b_s = b_s.merge(to_merge[['user_id', 'seller_id', 'last_click']], how='left', on=['user_id', 'seller_id'])

In [143]:
to_merge = train[train.action == 'cart'].groupby(['user_id', 'seller_id']).max()[['vtime']]
to_merge['last_cart'] = dt.datetime(2013, 8, 1) - to_merge.vtime
to_merge['last_cart'] = to_merge.last_cart.apply(lambda x: x.days)
to_merge = to_merge.reset_index()
b_s = b_s.merge(to_merge[['user_id', 'seller_id', 'last_cart']], how='left', on=['user_id', 'seller_id'])

In [144]:
b_s['last_purchase'] = b_s.last_purchase.fillna(121)
b_s['last_click'] = b_s.last_click.fillna(121)
b_s['last_cart'] = b_s.last_cart.fillna(121)

### sentiment

In [63]:
sentiment = pd.read_csv('reviewscore.csv')
to_merge = sentiment[['user_id', 'seller_id', 'sentimentscore_pos', 'sentimentscore_neu', 'sentimentscore_neg']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s[['sentimentscore_pos', 'sentimentscore_neu', 'sentimentscore_neg']] = b_s[['sentimentscore_pos', 'sentimentscore_neu', 'sentimentscore_neg']].fillna(0)

### outcome

In [72]:
g = out.groupby(['user_id', 'seller_id', 'action']).count()
g = g.reset_index()
to_merge = g[g.action == 'alipay'][['user_id', 'seller_id', 'item_id']]
to_merge['item_id'] = np.ones(len(to_merge))

In [73]:
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'outcome'})
b_s['outcome'] = b_s.outcome.fillna(0)

In [75]:
b_s.to_csv('b_s_train.csv', index=False)